# tensorboard

In [5]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def add_layer(inputs, in_size, out_size,n_layer, activation_function=None):
    layer_name = 'layer%s'%n_layer 
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]))
            tf.summary.histogram(layer_name+'/weights',Weights)
        with tf.name_scope('bias'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
            tf.summary.histogram(layer_name+'/biases',biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs

# Make up some real data
x_data = np.linspace(-1, 1, 300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

##plt.scatter(x_data, y_data)
##plt.show()

# define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 1],name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1],name='y_input')
# add hidden layer
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu,n_layer=1)
# add output layer
prediction = add_layer(l1, 10, 1, activation_function=None,n_layer=2)

# the error between prediction and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction), reduction_indices=[1]))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
# important step
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('graphsum',sess.graph)
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
    if i%50 == 0:
        result = sess.run(merged, 
                         feed_dict = {xs:x_data, ys:y_data})
        writer.add_summary(result, i)

InvalidArgumentError: You must feed a value for placeholder tensor 'inputs_1/x_input' with dtype float and shape [?,1]
	 [[Node: inputs_1/x_input = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'inputs_1/x_input', defined at:
  File "/Users/xianglongtan/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/xianglongtan/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/Users/xianglongtan/anaconda3/lib/python3

# 名称域（Name scoping）和节点（Node）  
典型的 TensorFlow 可以有数以千计的节点，如此多而难以一下全部看到，甚至无法使用标准图表工具来展示。为简单起见，我们为变量名划定范围，并且可视化把该信息用于在图表中的节点上定义一个层级。默认情况下， 只有顶层节点会显示。下面这个例子使用tf.name_scope在hidden命名域下定义了三个操作

In [5]:
import tensorflow as tf
with tf.name_scope('hidden') as scope:
    a = tf.constant(5, name='alpha')
    W = tf.Variable(tf.random_uniform([1,2],-1.0,1.0), name='weights')
    b = tf.Variable(tf.zeros([1],name='biases'))

TensorFlow 图表有两种连接关系：数据依赖和控制依赖。数据依赖显示两个操作之间的tensor流程，用实心箭头指示，而控制依赖用点线表示。在已展开的视图(上面的右图)中，除了用点线连接的CheckNumerics和control_dependency之外，所有连接都是数据依赖的。

In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
def conv_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([5,5,channels_in,channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding='SAME')
    act = tf.nn.relu(conv + b)
    return act

def fc_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([channels_in,channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    act = tf.nn.relu(tf.matmul(input,w)+b)
    return act

x = tf.placeholder(tf.float32, shape=[None,784])
y = tf.placeholder(tf.float32, shape=[None,10])
x_image = tf.reshape(x,[-1,28,28,1 ])

conv1 = conv_layer(x_image,1,32)
pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

conv2 = conv_layer(pool1,32,64)
pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
flattened = tf.reshape(pool2, [-1,7*7*64])

fc1 = fc_layer(flattened, 7*7*64, 1024)
logits = fc_layer(fc1, 1024, 10)

cross_entropy = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

sess = tf.Session()
sess.run(tf.initialize_all_variables())

for i in range(100):
    batch = mnist.train.next_batch(500)
    if i%10 == 0:
        [train_accuracy] = sess.run([accuracy], feed_dict={x:batch[0],y:batch[1]})
        print('step {0}, training accuracy {1}'.format(i,train_accuracy))
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1]})

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1080000028014183
step 10, training accuracy 0.07199999690055847
step 20, training accuracy 0.0860000029206276
step 30, training accuracy 0.10999999940395355
step 40, training accuracy 0.10199999809265137
step 50, training accuracy 0.10000000149011612
step 60, training accuracy 0.09000000357627869
step 70, training accuracy 0.09200000017881393
step 80, training accuracy 0.0860000029206276
step 90, training accuracy 0.1120000034570694


In [7]:
writer = tf.summary.FileWriter('graph950')
writer.add_graph(sess.graph)


In [31]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
def conv_layer(input, channels_in, channels_out,name='conv'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5,5,channels_in,channels_out]),name='W')
        b = tf.Variable(tf.zeros([channels_out]),name='B')
        conv = tf.nn.conv2d(input, w, strides=[1,1,1,1], padding='SAME')
        act = tf.nn.relu(conv + b)
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)
        tf.summary.histogram('activation',act)
        return tf.nn.max_pool(act, ksize = [1,2,2,1], strides=[1,2,2,1],padding='SAME')

def fc_layer(input, channels_in, channels_out, name='fc'):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([channels_in, channels_out]), name='W')
        b = tf.Variable(tf.zeros([channels_out]),name='B')
        return tf.nn.relu(tf.matmul(input, w)+b)

    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [47]:
x = tf.placeholder(tf.float32, shape=[None,784],name='x')
x_image = tf.reshape(x,[-1,28,28,1])
y = tf.placeholder(tf.float32, shape=[None,10],name='labels')

conv1 = conv_layer(x_image,1,32,'conv1')
conv2 = conv_layer(conv1,32,64,'conv2')

flattened = tf.reshape(conv2, [-1,7*7*64])
fc1 = fc_layer(flattened, 7*7*64,1024,'fc1')
logits = fc_layer(fc1,1024,10,'fc2')

In [38]:
with tf.name_scope('xent'):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(xent)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [39]:
tf.summary.scalar('cross_entropy',xent)
tf.summary.scalar('accuracy',accuracy)
tf.summary.image('input',x_image,3)

<tf.Tensor 'input_4:0' shape=() dtype=string>

In [46]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter('graph1055')
writer.add_graph(sess.graph)

for i in range(100):
    batch = mnist.train.next_batch(100)
    if i%5 == 0:
        s = sess.run(merged_summary, feed_dict={x:batch[0], y:batch[1]})
        writer.add_summary(s,i)
    sess.run(train_step,feed_dict={x:batch[0], y:batch[1]})



        

Instructions for updating:
Use `tf.global_variables_initializer` instead.


InvalidArgumentError: You must feed a value for placeholder tensor 'labels_2' with dtype float and shape [?,10]
	 [[Node: labels_2 = Placeholder[dtype=DT_FLOAT, shape=[?,10], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'labels_2', defined at:
  File "/Users/xianglongtan/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/xianglongtan/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/zmq/e

In [27]:
import os
import os.path
import shutil
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
def conv_layer(input, size_in, size_out, name="conv"):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram("activations", act)
    return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


def fc_layer(input, size_in, size_out, name="fc"):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    act = tf.matmul(input, w) + b
    tf.summary.histogram("weights", w)
    tf.summary.histogram("biases", b)
    tf.summary.histogram("activations", act)
    return act

In [4]:
import tensorflow as tf
tf.reset_default_graph()
xs = tf.placeholder(tf.float32,[None,1],name='x')
ys = tf.placeholder(tf.float32,[None,1],name='y')
with tf.name_scope('weight430'):
    w = tf.Variable(tf.truncated_normal([1]), name='w')
    
with tf.name_scope('loss'):
    y = xs*w
    loss = tf.reduce_mean(tf.square(y-ys))
        
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

tf.summary.histogram('weights430',w)
tf.summary.scalar('loss',loss)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('graph430')
    writer.add_graph(sess.graph)
    merged = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        if i%5 == 0:
            summ = sess.run(merged,feed_dict={xs:x_data,ys:y_data})
            writer.add_summary(summ,i)
            print('{0} round loss: {1}'.format(i,sess.run(loss,feed_dict={xs:x_data,ys:y_data})))
        sess.run(train_step,feed_dict={xs:x_data,ys:y_data})


0 round loss: 8.84542179107666
5 round loss: 6.6596503257751465
10 round loss: 5.0776896476745605
15 round loss: 3.9327380657196045
20 round loss: 3.104074001312256
25 round loss: 2.5043249130249023
30 round loss: 2.070253849029541
35 round loss: 1.7560930252075195
40 round loss: 1.5287176370620728
45 round loss: 1.3641537427902222
50 round loss: 1.2450497150421143
55 round loss: 1.1588479280471802
60 round loss: 1.096459150314331
65 round loss: 1.0513049364089966
70 round loss: 1.0186244249343872
75 round loss: 0.9949716329574585
80 round loss: 0.9778527617454529
85 round loss: 0.965462863445282
90 round loss: 0.9564957618713379
95 round loss: 0.9500057101249695


In [2]:
import numpy as np
x_data = np.random.rand(100).astype(np.float32).reshape([100,1]).astype(np.float32)
noise = np.random.normal(0,1,100).reshape([100,1])
y_data = (np.add(x_data*5,noise)).reshape([100,1]).astype(np.float32)


(100,)